# Análise exploratória de dados

In [79]:
import pandas as pd
import matplotlib as plt
import numpy as np
import seaborn as sns

In [80]:
df = pd.read_csv('../data/raw/data.csv')
dictionary = pd.read_csv('../data/external/dictionary.csv')

In [81]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## 1. Descrição dos dados

### Dicionário de Dados

In [82]:
dictionary

,variavel,descricao,tipo,subtipo
0,total_bill,Valor da conta em dólar,quantitativa,continua
1,tip,Valor da gorjeta em dolár,quantitativa,continua
2,sex,Sexo do pagador da conta,qualitativa,nominal
3,smoker,Fumante na festa,qualitativa,nominal
4,day,Dia da semana,qualitativa,nominal
5,time,Hora do dia,qualitativa,nominal
6,size,Tamanho da festa,quantitativa,discreta


In [83]:
df.dtypes

total_bill    float64
tip           float64
sex            object
smoker         object
day            object
time           object
size            int64
dtype: object

In [84]:
df.count()

total_bill    244
tip           244
sex           244
smoker        244
day           244
time          244
size          244
dtype: int64

### Dados faltantes:

In [85]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

#### Há 0 dados faltantes no dataset.

---

### Verificando linhas duplicadas:

In [86]:
df.duplicated().sum()

1

#### Removendo linhas duplicadas

In [87]:
df = df.drop_duplicates()

In [88]:
df.duplicated().sum()

0

---

In [89]:
df.describe(include='all')

,total_bill,tip,sex,smoker,day,time,size
count,243.000000,243.000000,243,243,243,243,243.000000
unique,NaN,NaN,2,2,4,2,NaN
top,NaN,NaN,Male,No,Sat,Dinner,NaN
freq,NaN,NaN,157,151,87,176,NaN
mean,19.813868,3.002387,NaN,NaN,NaN,NaN,2.572016
std,8.910071,1.385002,NaN,NaN,NaN,NaN,0.952356
min,3.070000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,13.380000,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,17.810000,2.920000,NaN,NaN,NaN,NaN,2.000000
75%,24.175000,3.575000,NaN,NaN,NaN,NaN,3.000000


- O sexo que mais compra são os homens
- O dia mais movimentado é o sabado
- O horário mais movimentado é a noite

## 2. Perguntas de partida e hipóteses

## 3. Insights